In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Bitcoin core regtest tx tester
This notebook will start up bitcoind in regtest mode, create transactions using python scripts, and broadcast them to validate that they work. Four transactions are tested here:<br><br>

- tx1 initial tx to fund customer. 1 output 
    - Send 11.3 btc to a P2SH-P2WPKH address. This will be used to fund the customer.
- tx2 funding tx with change. 2 outputs
    - 10.2 btc to an escrow (2-of-2 multisig) 
    - 1 btc to a change address
- tx3 merch close. 1 output
    - 10.1 btc to p2wsh with two ways to spend from it:
        - 2 of 2 multisig signed by cust_pk and merch_pk (cust-close-tx)
        - to_self_delay to merch_close_pk (merch-claim-tx)

Funding private key is used to fund the escrow tx. <br> 
Merch private key and Cust private key are used to generate the pubkeys in the escrow tx. <br>
The Cust private key is also used for the P2WPKH change output.

In [3]:
funding_privkey_hex = "1111111111111111111111111111111111111111111111111111111111111111"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

merch_privkey_hex = "2111111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

change_privkey_hex = "4111111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_close_privkey_hex = "5111111111111111111111111111111111111111111111111111111111111111"
merch_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_close_privkey_hex)).hex()

Start up regtest mode, delete any history so we are starting from scratch.
Mine 101 blocks

In [4]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [5]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2NEp346GsvyPAu3X9oKGjrzLXc8vw8CqG7T


Send btc to the escrow funder

In [6]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 11.3")
print(txid_1)

a70d6a7978107782995aeef41e0227bc5d3c9e5a0a17ede9d1202f1c94a49f53


In [7]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 1


In [8]:
os.system("bitcoin-cli -regtest generate 1");

Generate raw escrow funding transaction

In [9]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_change.py" 
                        + " --funding_privkey " + funding_privkey_hex
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --amount_btc " + "11.3"
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --change_pubkey " + change_pubkey_hex
                        + " --escrow_btc " + "10.2"
                        + " --change_btc " + "1")
print(raw_escrow_tx)

02000000000101539fa4941c2f20d1e9ed170a5a9e3c5dbc27021ef4ee5a9982771078796a0da70100000017160014fc7250a211deddc70ee5a2738de5f07817351cefffffffff02fff6cb3c000000002200201f3abc4d03a3b61defffe4d163f25fcd0edc96f352ed1324385dc1ce28debd0200e1f50500000000160014bb524cfc4313630bdd51106c1a1b1fb58a3cea3c02483045022100cf3e1dd23f336bab63b9a9f7738592e172a270765c3172bfb352e7cc2a41f2560220663eca88758ea0c08f24988ce90d2faeedb78691dd150aaad840f5c87235111b0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


Broadcast escrow funding transaction

In [10]:
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(escrow_txid)

5eb0c50e6f725b88507cda84f339aba539bc99853436db610d6a476a207f82d9


In [11]:
raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [12]:
os.system("bitcoin-cli -regtest generate 1");

Create transaction spending from the change output of the escrow transaction

In [13]:
raw_merch_close_tx = subprocess.getoutput("python merch_close.py"        
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex      
                + " --merch_close_pubkey " + merch_close_pubkey_hex   
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output       
                + " --txid " + escrow_txid
                + " --index " + "0"
                + " --amount_btc " + "10.2"
                + " --output_btc " + "10.1")
print(raw_merch_close_tx)

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_close_tx)
# print(decoded)

02000000000101d9827f206a476a0d61db36348599bc39a5ab39f384da7c50885b726f0ec5b05e0000000000ffffffff018060333c000000002200204de4a2361c5f251e23b9aa799012a9c94131ab51ec4be0e2a9857125c375e19d04004730440220376822e162fbd4536e8322939e822f258473531a3f605a14d237a0b34891706402201d0e5c11eb0d43e09b036102aff7db6f368895f3901767e65628f958aaf524ff01483045022100bc85eddf9b5348f779d6a48835935b3bfc7ee4260babde887d18ec4ee2c64101022027fc7b6155a8a677194541ddb73a46b2477508c4947aeb7c5e4b9145df81683701475221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae00000000


In [14]:
merch_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_close_tx + " true")
print(merch_close_txid)

e2b72736d785639b7aab175245410bfc8aca89037a6e1c893a02bb11077b4872


In [15]:
merch_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_close_tx)
print(decoded)

{
  "txid": "e2b72736d785639b7aab175245410bfc8aca89037a6e1c893a02bb11077b4872",
  "hash": "22d16de791ce9664a6638238009abc3594c5a92978cd6636142acc287d465841",
  "version": 2,
  "size": 315,
  "vsize": 150,
  "weight": 597,
  "locktime": 0,
  "vin": [
    {
      "txid": "5eb0c50e6f725b88507cda84f339aba539bc99853436db610d6a476a207f82d9",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "30440220376822e162fbd4536e8322939e822f258473531a3f605a14d237a0b34891706402201d0e5c11eb0d43e09b036102aff7db6f368895f3901767e65628f958aaf524ff01",
        "3045022100bc85eddf9b5348f779d6a48835935b3bfc7ee4260babde887d18ec4ee2c64101022027fc7b6155a8a677194541ddb73a46b2477508c4947aeb7c5e4b9145df81683701",
        "5221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value"

In [16]:
raw_merch_claim_tx = subprocess.getoutput("python merch_claim.py"   
                + " --merch_pubkey " + merch_pubkey_hex   
                + " --cust_pubkey " + cust_pubkey_hex   
                + " --merch_close_privkey " + merch_close_privkey_hex   
                + " --output_pubkey " + "024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1"    
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output         
                + " --txid " + merch_close_txid
                + " --index " + "0"
                + " --amount_btc " + "10.1"
                + " --output_btc " + "10")
print(raw_merch_claim_tx)


decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_claim_tx)
print(decoded)

0200000000010172487b0711bb023a891c6e7a0389ca8afc0b41455217ab7a9b6385d73627b7e20000000000cf0500000100ca9a3b00000000160014b7cfef435e3701fdedb7a11164ae44d561698bb9034830450221008ff269ceb1e2925252a9c05badb70ef1ab3b274a002b45a7afd53cb45efe39410220353c836c6b6cb151b0660b3bfdb7de739521bfbddc435f7f04bf2af480e58439010072635221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae6702cf05b2752102ab573100532827bd0e44b4353e4eaa9c79afbc93f69454a4a44d9fea8c45b5afac6800000000
{
  "txid": "94683cb4cba2bb647c5643206fb5644d8d95e364a763dd0342361dd6469d6294",
  "hash": "92dbfbb800f7d5b0c4b695be18b1def10a15ecd585d0332173057b68b07b7a48",
  "version": 2,
  "size": 274,
  "vsize": 130,
  "weight": 520,
  "locktime": 0,
  "vin": [
    {
      "txid": "e2b72736d785639b7aab175245410bfc8aca89037a6e1c893a02bb11077b4872",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
      

In [17]:
os.system("bitcoin-cli -regtest generate 1486");

Only 1486 blocks mined, one less than the delay_to_self of 1487 blocks

In [18]:
merch_claim_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_claim_tx + " true")
print(merch_claim_txid)

error code: -26
error message:
non-BIP68-final (code 64)


Generate 1 more block and try again

In [19]:
os.system("bitcoin-cli -regtest generate 1");

In [20]:
merch_claim_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_claim_tx + " true")
print(merch_claim_txid)

94683cb4cba2bb647c5643206fb5644d8d95e364a763dd0342361dd6469d6294


In [21]:
merch_claim_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_claim_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_claim_tx)
print(decoded)

{
  "txid": "94683cb4cba2bb647c5643206fb5644d8d95e364a763dd0342361dd6469d6294",
  "hash": "92dbfbb800f7d5b0c4b695be18b1def10a15ecd585d0332173057b68b07b7a48",
  "version": 2,
  "size": 274,
  "vsize": 130,
  "weight": 520,
  "locktime": 0,
  "vin": [
    {
      "txid": "e2b72736d785639b7aab175245410bfc8aca89037a6e1c893a02bb11077b4872",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30450221008ff269ceb1e2925252a9c05badb70ef1ab3b274a002b45a7afd53cb45efe39410220353c836c6b6cb151b0660b3bfdb7de739521bfbddc435f7f04bf2af480e5843901",
        "",
        "635221024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c52ae6702cf05b2752102ab573100532827bd0e44b4353e4eaa9c79afbc93f69454a4a44d9fea8c45b5afac68"
      ],
      "sequence": 1487
    }
  ],
  "vout": [
    {
      "value": 10.00000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 b7c

In [22]:
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 